<a href="https://colab.research.google.com/github/nikbearbrown/INFO_7390_Art_and_Science_of_Data/blob/main/TA_Contest_Fall_23/ShardulChavan/Week8-Generating-Text/Text_Generation_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing Libraries

In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.3 MB/s eta 0:00:00


### Importing Libraries
The library **transformers** has pre-trained AI models such as GPT2LMHeadModel, GPT2Tokenizer for various language tasks  
### Loading Pre-trained Model and Tokenizer:
The code loads a pre-trained AI model called "gpt2" and a tokenizer. The model is like a smart writer, and the tokenizer helps the model understand and process text.

###Preparing the Model:
The model is set to "generate text" mode using model.eval(). It's like telling the model to start writing.

###Setting Up the Prompt:
The code provides a starting sentence called a "prompt." The model will use this to continue generating text.

###Encoding the Prompt:
The starting sentence is turned into numbers that the model can understand. This is done using the tokenizer, which converts words into codes.

### Generating Text:
The model starts generating text based on the prompt. The generate function is used, which takes the encoded prompt and generates new text. It follows a strategy called "temperature" (0.5 in this case) to control how random the text should be.

### Decoding results:
The generated text is in numbers, so it's decoded back into words using the tokenizer. The resulting text is printed, which is the AI's creative writing based on the prompt.

### Removing Repeated Phrases:
To make the generated text more interesting, a post-processing step removes repeated phrases. This helps the text sound more natural and varied.

### Final Output:
The code prints the final generated text that the AI created based on the given prompt.

In [24]:
# Importing Libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set the model for generating text
model.eval()

# Prompt for text generation
prompt = "In a world where technology advanced beyond imagination"

# Encode the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt")


temperature = 0.5  # temperature for randomness of the text
max_length = 100
attention_mask = input_ids.new_ones(input_ids.shape)
pad_token_id = tokenizer.eos_token_id
output = model.generate(input_ids, attention_mask=attention_mask, pad_token_id=pad_token_id, max_length=max_length, num_return_sequences=1, temperature=temperature)

# Decode and print generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Apply post-processing to remove repeated phrases
def remove_repeated_phrases(text):
    phrases = text.split(". ")
    unique_phrases = []
    for phrase in phrases:
        if phrase not in unique_phrases:
            unique_phrases.append(phrase)
    return ". ".join(unique_phrases)

generated_text = remove_repeated_phrases(generated_text)

# Final Output
print("Generated Text:")
print(generated_text)


Generated Text:
In a world where technology advanced beyond imagination, the world of the future is a world of possibilities.

The future is not a future where we can make a living, but a future where we can make a living.

The future is a world where we can make a living.

The future is a world where we can make a living.

The future is a world where we can make a living.

The future is a world where we can make a living


In this case, the model seems to get stuck in a loop where it repeats the same phrases over and over again, resulting in a lack of meaningful variation in the text.

## What's Wrong:

**Repetitive Phrasing:** The phrases "The future is a world where we can make a living" and similar variations are repeated multiple times, creating a repetitive and uninteresting pattern.

**Lack of Meaningful Variation:** The text lacks meaningful variation and progression. The repetition doesn't contribute to the development of ideas or narrative.

**Contextual Disconnection:** The text appears to lose its context as it progresses, causing the repetition to become more noticeable and disconnected from the initial theme.

## What could be the possible causes:

**Seed Phrase Influence:** The initial prompt might be influencing the repetitive output, especially if the training data contained similar patterns.

**Temperature Setting:** The temperature parameter might be set low, making the model overly deterministic and leading to repeated choices.

**Limited Training Data Variation:** If the training data lacks diverse and creative examples, the model might struggle to generate unique content. (But this couldn't be the reason in our case)

## How to Improve:

**Adjust Temperature**: We could try to experiment with a higher temperature setting to introduce more randomness into the text generation process.

**Vary Prompts**: Also trying different prompts that encourage the model to explore various directions and concepts could be an option.

**Larger Beam Size:** Moreover we can consider using a diverse beam search method to reduce repetition and set larger beam size during generation to encourage more varied outputs.

Few additional approaches we can consider to further reduce repetition in generated text are:

1. N-gram Tracking: Keep track of generated n-grams (subsequences of length n) and use them to guide the generation process. Avoid generating n-grams that have appeared recently to promote diversity.

2. Denoising Autoencoders: Train a denoising autoencoder to recognize and penalize repeated phrases during text generation.

3. Sampling Strategy: Experiment with different sampling strategies (e.g., top-k, top-p sampling) to influence the distribution of generated tokens.

4. Custom Decoding Algorithm: Develop a custom decoding algorithm that incorporates uniqueness and diversity as objectives.


In this below snippet code compared to the previous one we have utilized the Diverse Beam Search. In the previous code, we used a simpler approach of temperature-controlled generation to produce a single text. While the temperature-controlled generation can introduce randomness, it doesn't explicitly tackle the issue of repetitiveness.

By incorporating Diverse Beam Search, we've addressed the problem of repetition more effectively. The no_repeat_ngram_size parameter helps avoid generating sequences with repeated n-grams, which leads to a richer and more diverse set of generated texts. This results in a better overall output quality by reducing monotony and enhancing creativity in the generated text.

In [27]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = "gpt2"
model_GPT2 = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set the model to evaluation mode
model_GPT2.eval()

# Prompt for text generation
prompt = "In a quaint village, a curious child discovered an ancient map"

# Encode the prompt
prompt_input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Diverse Beam Search parameters
beam_size = 5
max_length = 100
attention_mask = prompt_input_ids.new_ones(prompt_input_ids.shape)
pad_token_id = tokenizer.eos_token_id
# Generate diverse text using beam search
output_sequences = model_GPT2.generate(
    prompt_input_ids,
    attention_mask=attention_mask,
    pad_token_id=pad_token_id,
    max_length=max_length,
    num_return_sequences=beam_size,
    num_beams=beam_size,
    no_repeat_ngram_size=3  # Avoid repeated n-grams
)

# Decode and print generated text
generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in output_sequences]

print("Generated Texts:")
for i, text in enumerate(generated_texts):
    print(f"{i+1}. {text}")


Generated Texts:
1. In a quaint village, a curious child discovered an ancient map of the world.

It was a map of a world that had been lost for thousands of years. It was the only one of its kind in the world, and the only map that had ever been found in the history of the universe. The map had been discovered by a group of explorers who had traveled to the distant past to find the lost map. The group had found the map, but it had never been found.

2. In a quaint village, a curious child discovered an ancient map of the world.

It was a map of a world that had been lost for thousands of years. It was the only one of its kind in the world, and the only map that had ever been found in the history of the universe. The map had been discovered by a group of explorers who had traveled to the distant past to find the lost map. The group had found the map, but it had never been found. The
3. In a quaint village, a curious child discovered an ancient map of the world.

It was a map of a world

After experimenting with Temperature Setting + Prompt Variation, Temperature Setting + Post-Processing and repetition reduction technique, let's try Text Priming + Model Size in which we start the generation with a complete sentence to establish context and use a larger model variant for improved coherence and depth.

Moreover by using a complete sentence as the priming text, the model is more likely to produce coherent and relevant text that extends from that initial context. Additionally, the use of a larger model (gpt2-large) and longer max_length helps in generating more detailed and diverse text.

In [15]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load a larger pre-trained model and tokenizer
model_name = "gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set the model to generate text
model.eval()

# Text priming with a complete sentence
priming_text = "In a world where technology advanced beyond imagination"

# Encode the priming text
priming_input_ids = tokenizer.encode(priming_text, return_tensors="pt")

# Generate text with text priming
temperature=0.2
max_length = 150  # Longer length to provide depth to the generated content
attention_mask = priming_input_ids.new_ones(priming_input_ids.shape)
pad_token_id = tokenizer.eos_token_id
output = model.generate(priming_input_ids, attention_mask=attention_mask, pad_token_id=pad_token_id, max_length=max_length, num_return_sequences=1, temperature=temperature)


# Decode and print generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Text:")
print(generated_text)

Generated Text:
In a world where technology advanced beyond imagination, the idea of a "living" computer was a fantasy.

But in the early 1990s, the idea of a computer that could be "living" was a reality.

The first computer to be "living" was the IBM PC.

The PC was a computer that could be "living" by using a "living" operating system.

The operating system was called Windows.

Windows was a computer operating system that was based on the UNIX operating system.

The UNIX operating system was a Unix-like operating system that was based on the UNIX philosophy of "do one thing, do it well".

The UNIX philosophy was based on


In summary, the combination of a larger model, text priming, longer length, controlled randomness, avoidance of repeated n-grams, and advanced language patterns collectively contributed to the generation of better results.

**Using a Larger Pre-trained Model (gpt2-large):** Switching to a larger pre-trained model, such as gpt2-large, often leads to improved text generation. Larger models have more parameters and capacity to capture intricate patterns in the input data, resulting in more coherent and contextually relevant outputs.

**Text Priming: **By providing an initial context ("In a world where technology advanced beyond imagination"), you guide the model's generated text to follow a coherent direction from the given starting point. This helps the model stay on topic and produce text that logically extends from the priming text.

**Longer Maximum Length:** Increasing the max_length parameter to 150 allows the model to generate more extended and detailed text. This additional length enables the model to provide more context and depth to the generated content.

**Low Temperature (temperature=0.2):** A lower temperature value (0.2 in this case) reduces randomness in the generated text. This often results in more focused and deterministic output, which can contribute to more coherent and structured text.

**No Repeated N-grams**: The no_repeat_ngram_size parameter set to 3 prevents the generation of repeated n-grams. This helps avoid monotony and redundancy in the generated text, leading to more diverse and engaging content.

**Advanced Language Patterns:** The increased model size and capacity of gpt2-large enable the model to capture and replicate more complex language patterns, resulting in text that appears more sophisticated and well-structured.

Thus each of these factors plays a role in producing coherent, contextually relevant, and engaging generated text.